In [1]:
import pandas as pd
import MySQLdb
from datetime import datetime,timedelta

In [2]:
Anio='2019'
Mes='05'
Dia_act='21'

In [3]:
# nombre de los archivos Demografico
nom_actual_dem='..\..\..\BASES DATATOOLS\ASIGNACION\\2019\\MAYO\Sistema de cobro_'+Dia_act+'_'+Mes+'_'+Anio+'.xlsx'
nombre_hoja_dem='Demografico'

In [4]:
asignacion_actual_dem=pd.read_excel(open(nom_actual_dem,'rb'), sheet_name=nombre_hoja_dem, dtype={"IDENTIFICACION_DEUDOR": str,'TELEFONO':str})

In [5]:
asignacion_actual_dem['ASIGNADO']='SI'
asig_demo=asignacion_actual_dem[['TELEFONO','ASIGNADO']]

In [6]:
db2 = MySQLdb.connect(host="172.18.55.99",    # tu host, usualmente localhost
                     user="comandato",         # tu usuario
                     passwd="comandato123",  # tu password
                     db="call_center")        # el nombre de la base de datos

In [7]:
cur2= db2.cursor()

In [8]:
consulta="""SELECT id FROM call_center.campaign where queue in ('3333','3334');"""
campañas=pd.read_sql(consulta, con=db2)

In [9]:
campañas.head()

,id
0,913
1,914
2,924
3,926
4,927


In [10]:
pull=''
for campaña in campañas.itertuples():
    pull=pull+str(campaña.id)+','

In [11]:
campañas_tip='('+pull[:-1]+')'

In [12]:
campañas_tip

'(913,914,924,926,927,932,933)'

In [13]:
consulta2="SELECT  id,phone as 'TELEFONO' FROM call_center.calls where id_campaign in"+campañas_tip+ "and status is null;"
llamadas = pd.read_sql(consulta2, con=db2)

In [14]:
llamadas.head()

,id,TELEFONO
0,4074811,0999751760
1,4074813,0999699536
2,4074814,0999695345
3,4074815,0999693185
4,4074816,0999690720


In [15]:
verificacion = pd.merge(llamadas,asig_demo,on=['TELEFONO'],how='left')

In [16]:
verificacion.drop_duplicates(['id'], keep='first',inplace=True)

In [17]:
verificacion[verificacion['ASIGNADO']=='SI'].TELEFONO.count()

12371

In [18]:
eliminar=verificacion[verificacion['ASIGNADO'].isnull()]

In [19]:
eliminar

,id,TELEFONO,ASIGNADO
50,4074850,0999587216,NaN
238,4075019,0998867400,NaN
420,4075171,0997853250,NaN
427,4075177,0997829450,NaN
958,4075627,0995190619,NaN
1036,4075697,0994597649,NaN
1226,4075865,0993911809,NaN
1654,4076226,0991642586,NaN
1780,4076326,0991028172,NaN
2297,4076755,0986576762,NaN


In [20]:
cont=0
cont1=0
for row in eliminar.itertuples():
    delete1 = """ DELETE FROM `call_center`.`call_attribute` WHERE `id_call`='{0}'; """.format(row.id)
    try:
        cur2.execute(delete1)
        db2.commit()
        cont=cont+1
    except:
        db2.rollback()
    delete2 = """ DELETE FROM `call_center`.`calls` WHERE `id`='{}'; """.format(row.id)
    try:
        cur2.execute(delete2)
        db2.commit()
        cont1=cont1+1
    except:
        db2.rollback()
print(cont)
print(cont1)

61
61


In [21]:
nuevo_archivo='NUMEROS ELIMINADOS PREDICTIVA DATATOOLS.xlsx'
writer = pd.ExcelWriter(nuevo_archivo, engine='xlsxwriter')
eliminar.to_excel(writer,sheet_name='BASE')
worksheet = writer.sheets['BASE']
writer.save()